In [6]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe 핸드 모델 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 그리기 캔버스 초기화
canvas = None
# 현재 그리기 위치
drawing = False
# 마지막 인덱스 손가락 위치
last_x, last_y = 0, 0

cap = cv2.VideoCapture(0)

while True:
    success, frame = cap.read()
    if not success:
        break
    frame = cv2.flip(frame, 1)
    # 캔버스가 초기화되지 않았다면 현재 프레임 크기로 초기화
    if canvas is None:
        canvas = np.zeros_like(frame)

    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 손 감지 결과
    results = hands.process(rgb_frame)

    # 손 감지
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 인덱스 손가락 끝의 랜드마크 인덱스는 8번
            x = int(hand_landmarks.landmark[8].x * frame.shape[1])
            y = int(hand_landmarks.landmark[8].y * frame.shape[0])

            if drawing:
                # 이전 위치와 현재 위치 사이를 선으로 연결
                cv2.line(canvas, (last_x, last_y), (x, y), (255, 0, 0), 5)
            last_x, last_y = x, y
            drawing = True

    # 결과 화면에 그리기
    frame = cv2.add(frame, canvas)
    cv2.imshow('Hand Tracking Drawing', frame)

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()